Importa o modelo a ser fatiado e mapeado

In [1]:
import matplotlib.pyplot as plt
import gui
import files
from timer import Timer
from components import layer
from os import getcwd

home = getcwd()
folders = files.Paths(home)
path_input, file_name = gui.load_model(folders)
[dpi, layer_height] = gui.ask_parameters_input()
if file_name.endswith(".stl") or file_name.endswith(".STL"):
    folders.create_layers_3d(path_input, dpi, layer_height)
else:
    folders.create_layers_2d(path_input, dpi, layer_height)

layer.divide_islands(folders)

plt.rcParams["figure.figsize"]=5,5
#TODO: encontrar uma forma de jogar layerHight para o slicing W/images (POSSIVELMENTE COM O RUNSINGLEMODEL)
# TODO: retirar todos os parametros que nao uso
# TODO: dar o Typing em todos

Paredes Finas

In [1]:
import matplotlib.pyplot as plt
import gui
import files
from timer import Timer
from components import layer
from os import getcwd
try:
    folders
except NameError:
    home = getcwd()
    folders = files.Paths(home)

[nozzle_diam_internal,nozzle_diam_external] = gui.ask_parameters_thin_walls()
# from components.layer import Layer
camadas = folders.list(layers=1)
for i,c in enumerate(camadas):
    camada = folders.load_layer_json(c)
    with Timer("Fase 1: Camada: " + str(i)):
        camada.make_thin_walls(folders, nozzle_diam_external, nozzle_diam_internal)
    
# TODO: arvore de paredes finas
# TODO: quando duas áreas se encontram
# TODO: limite de parede diam minimo
# TODO: anotar todos os (não fechou contorno)
# TODO arrumar para origens extremamente pequenas, ignorar da problema
# TODO arrumar casos onde a beirada esta vazndo criando areas triangulares distantes
# FIXME: Precisa mesmo separar linha 1 2 topo e baixo? muitos elementos estão sendo criados

Fase 1: Camada: 0
Criando paredes finas
OK: fechou contorno
OK: fechou contorno
OK: fechou contorno
Retirando Paredes finas da camada
Total de [Retirando Paredes finas da camada] : 0.3417036533355713
Total de [Criando paredes finas] : 17.665004014968872
Total de [Fase 1: Camada: 0] : 17.665154218673706


Contornos

In [ ]:
import matplotlib.pyplot as plt
import gui
import files
from timer import Timer
from components import layer
from os import getcwd
try:
    folders
except NameError:
    home = getcwd()
    folders = files.Paths(home)

[void_max,internal_max,external_max] = gui.ask_parameters_offsets()
# from components.layer import Layer
camadas = folders.list(layers=1)
for i,c in enumerate(camadas):
    camada = folders.load_layer_json(c)
    with Timer("Fase 1: Camada: " + str(i)):
        camada.make_offsets(void_max, external_max, internal_max)

Fase 1: Camada: 0
Total de [Fase 1: Camada: 0] : 0.00013971328735351562


AttributeError: 'Layer' object has no attribute 'make_offsets'